### _onm_df_20210128_20210531.csv
### _sensor_df_20210128_20210531_p1_1hour.csv
### _sensor_df_20210128_20210531_p1_10mins.csv

In [2]:
from pandas import read_csv
import pandas as pd
import numpy as np
import openpyxl
import os
import fnmatch

print(os.getcwd())
date_start = '20210129'
date_end   = '20210531'
save_name  =  date_start+'_'+date_end
base_dir   = 'C:/elsys/solar/'
RSRSID_list=['RSRS0000000239', 'RSRS0000000241', 'RSRS0000000247', 'RSRS0000000249']

C:\Users\VISLAB_PHY\Workspace\JUPYTER


# SENSOR (API)
#### # 시간단위 / 10분단위로 mean 치기

In [125]:
# SENSOR csv 통합
sensor_path = base_dir+'_SENSOR/csv/'
time_list       = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24']

file_list   = os.listdir(sensor_path)
print(len(file_list))
sensor_df = pd.DataFrame()

for filename in file_list:
    #if (filename[:-4] not in err_date_list):
    #print("{}<={}".format(filename[:-4],date_end))
    if ((filename[:-4]>=date_start) & (filename[:-4]<=date_end)):
        newdata = pd.read_csv(sensor_path+'/'+filename)
        
        
        #print(filename, end='')
        sensor_df = pd.concat([sensor_df, newdata])
        sensor_df.sort_values('TIME')

display(sensor_df)
sensor_df.to_csv(sensor_path+"/_sensor_df_"+save_name+".csv",mode='w',index=False)

128


,id,loc_oid,TIME,solarradiation,uv,temp,humidity,winddir,windspeed,windgust,dewpoint,maxdailygust,feelslike,hourlyrainin,dailyrainin,weeklyrainin,monthlyrainin,yearlyrainin
0,63,1,2021-01-29 00:08:00,0.0,0,-4.722,77.0,0,1.609,3.308,-8.128,10.013,-7.339,0.0,0.0,12.192,12.192,12.192
1,64,1,2021-01-29 00:18:00,0.0,0,-4.778,73.0,301,3.979,7.778,-8.867,10.013,-10.133,0.0,0.0,12.192,12.192,12.192
2,65,1,2021-01-29 00:28:00,0.0,0,-4.778,74.0,10,2.995,4.381,-8.689,10.013,-9.233,0.0,0.0,12.192,12.192,12.192
3,66,1,2021-01-29 00:38:00,0.0,0,-4.778,73.0,318,2.012,3.308,-8.867,10.013,-8.039,0.0,0.0,12.192,12.192,12.192
4,67,1,2021-01-29 00:48:00,0.0,0,-4.722,73.0,331,3.397,5.498,-8.811,10.013,-9.561,0.0,0.0,12.192,12.192,12.192
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,17768,1,2021-05-31 23:17:00,0.0,0,13.222,85.0,96,0.000,0.000,10.756,5.498,13.222,0.0,0.0,0.000,96.520,502.412
140,17769,1,2021-05-31 23:27:00,0.0,0,13.111,86.0,117,0.000,0.000,10.822,5.498,13.111,0.0,0.0,0.000,96.520,502.412
141,17770,1,2021-05-31 23:37:00,0.0,0,13.000,88.0,155,0.000,0.000,11.056,5.498,13.000,0.0,0.0,0.000,96.520,502.412
142,17771,1,2021-05-31 23:47:00,0.0,0,13.000,89.0,129,0.000,0.000,11.228,5.498,13.000,0.0,0.0,0.000,96.520,502.412


In [126]:
'''
1시간 간격 csv 파일 생성
#일시 패턴 변환(2019-08-20 5:00 -> 2019082005)
sensor_df['NEW_TIME'] = sensor_df['TIME'].str[0:4]+sensor_df['TIME'].str[5:7]+sensor_df['TIME'].str[8:10]+sensor_df['TIME'].str[11:13]+sensor_df['TIME'].str[-5:-4].replace(" ","0")+sensor_df['TIME'].str[-4:-3]
sensor_df = sensor_df[(sensor_df['NEW_TIME'].astype(str).str[0:10]>=date_start) & (sensor_df['NEW_TIME'].astype(str).str[0:10]<date_end)]
#sensor_df = sensor_df.interpolate(method='linear')# 결측값 보간
sensor_df.insert(0, 'DATE', sensor_df['NEW_TIME'].values, True)

# drop columns
sensor_df.drop(['id','loc_oid', 'TIME', 'NEW_TIME'], axis=1, inplace=True)
sensor_df['DATE'] = sensor_df['DATE'].astype('str')
sensor_df.to_csv(base_dir+"/_sensor_df_"+save_name+"_p1_10mins.csv",mode='w',index=False)
display(sensor_df)

step = 6
hourlysensor = []
sensor_df['DATE'] = sensor_df['DATE'].astype('float')
for i in range(0, sensor_df.shape[0], step):
    hourlysensor.append(np.max(sensor_df[i:i+step]))
    #print('{}:{}\t'.format(i, i+step), end='')
hourlysensor_df = pd.DataFrame(hourlysensor,columns=sensor_df.columns)
hourlysensor_df['DATE'] = hourlysensor_df['DATE'].astype('str').str[0:10]
hourlysensor_df.sort_values('DATE')

hourlysensor_df.to_csv(base_dir+"/_sensor_df_"+save_name+"_p1_1hour.csv",mode='w',index=False)
display(hourlysensor_df)    
'''

,DATE,solarradiation,uv,temp,humidity,winddir,windspeed,windgust,dewpoint,maxdailygust,feelslike,hourlyrainin,dailyrainin,weeklyrainin,monthlyrainin,yearlyrainin
0,202101290008,0.0,0,-4.722,77.0,0,1.609,3.308,-8.128,10.013,-7.339,0.0,0.0,12.192,12.192,12.192
1,202101290018,0.0,0,-4.778,73.0,301,3.979,7.778,-8.867,10.013,-10.133,0.0,0.0,12.192,12.192,12.192
2,202101290028,0.0,0,-4.778,74.0,10,2.995,4.381,-8.689,10.013,-9.233,0.0,0.0,12.192,12.192,12.192
3,202101290038,0.0,0,-4.778,73.0,318,2.012,3.308,-8.867,10.013,-8.039,0.0,0.0,12.192,12.192,12.192
4,202101290048,0.0,0,-4.722,73.0,331,3.397,5.498,-8.811,10.013,-9.561,0.0,0.0,12.192,12.192,12.192
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,202105302316,0.0,0,13.000,92.0,124,0.000,0.000,11.728,5.498,13.000,0.0,0.0,0.000,96.520,502.412
140,202105302326,0.0,0,12.889,92.0,32,0.000,0.000,11.622,5.498,12.889,0.0,0.0,0.000,96.520,502.412
141,202105302336,0.0,0,12.889,92.0,192,0.000,0.000,11.622,5.498,12.889,0.0,0.0,0.000,96.520,502.412
142,202105302346,0.0,0,12.889,91.0,136,0.000,0.000,11.456,5.498,12.889,0.0,0.0,0.000,96.520,502.412


,DATE,solarradiation,uv,temp,humidity,winddir,windspeed,windgust,dewpoint,maxdailygust,feelslike,hourlyrainin,dailyrainin,weeklyrainin,monthlyrainin,yearlyrainin
0,2021012900,0.0,0.0,-4.722,77.0,331.0,3.979,7.778,-8.128,11.086,-7.339,0.0,0.0,12.192,12.192,12.192
1,2021012901,0.0,0.0,-4.889,85.0,353.0,4.604,6.705,-7.394,11.086,-5.389,0.0,0.0,12.192,12.192,12.192
2,2021012902,0.0,0.0,-5.278,78.0,358.0,4.381,8.896,-8.500,11.086,-5.278,0.0,0.0,12.192,12.192,12.192
3,2021012903,0.0,0.0,-5.500,78.0,357.0,4.113,5.498,-8.717,11.086,-5.500,0.0,0.0,12.192,12.192,12.192
4,2021012904,0.0,0.0,-5.778,76.0,351.0,2.995,5.498,-9.317,11.086,-9.194,0.0,0.0,12.192,12.192,12.192
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2916,2021053020,0.0,0.0,16.278,80.0,149.0,0.000,0.000,11.633,5.498,16.278,0.0,0.0,0.000,96.520,502.412
2917,2021053021,0.0,0.0,14.222,87.0,151.0,0.000,0.000,11.489,5.498,14.222,0.0,0.0,0.000,96.520,502.412
2918,2021053022,0.0,0.0,13.278,92.0,204.0,0.000,0.000,11.839,5.498,13.278,0.0,0.0,0.000,96.520,502.412
2919,2021053023,0.0,0.0,13.000,92.0,192.0,0.000,0.000,11.728,5.498,13.000,0.0,0.0,0.000,96.520,502.412


# ONM (API)
파일 통합은 JsonToCsv에서 이미 함

In [3]:
# ONM csv 통합
onm_path = base_dir+'_ONM/csv_'+RSRSID_list[2]

file_list   = os.listdir(onm_path)
print(len(file_list))
onm_df = pd.DataFrame()

for filename in file_list:
    if fnmatch.fnmatch(filename, '_*_*.csv'):
        onm_df = pd.read_csv(onm_path+'/'+filename, encoding='CP949')
        print(filename)
        break;

print(onm_df.columns)
display(onm_df)
onm_df.to_csv(base_dir+"/_"+RSRSID_list[2]+"onm_df_"+save_name+"_daily.csv",mode='w',index=False, encoding='CP949')

# 설비용량
capacity = onm_df['CAPACITY'].iloc[0]
print(capacity)


onm_df['NEW_DATE'] = onm_df['PATN_DT'].str[0:4]+onm_df['PATN_DT'].str[5:7]+onm_df['PATN_DT'].str[8:10]
onm_df = onm_df[(onm_df['NEW_DATE'].astype(str).str[0:10]>=date_start) & (onm_df['NEW_DATE'].astype(str).str[0:10]<=date_end)]
onm_df.insert(0, 'DATE', onm_df['NEW_DATE'].values, True)
onm_df = onm_df.drop(['COMPANY_NM', 'SET_GEN_NM', 'GEN_ID', 'GEN_NM', 'CBP_GEN_ID','CAPACITY','NEW_DATE','PATN_DT'], axis=1)

display(onm_df)

516
_20200101_20210531.csv
Index(['COMPANY_NM', 'SET_GEN_NM', 'GEN_ID', 'GEN_NM', 'CBP_GEN_ID',
       'CAPACITY', 'PATN_DT', 'HR_01', 'HR_02', 'HR_03', 'HR_04', 'HR_05',
       'HR_06', 'HR_07', 'HR_08', 'HR_09', 'HR_10', 'HR_11', 'HR_12', 'HR_13',
       'HR_14', 'HR_15', 'HR_16', 'HR_17', 'HR_18', 'HR_19', 'HR_20', 'HR_21',
       'HR_22', 'HR_23', 'HR_24'],
      dtype='object')


,COMPANY_NM,SET_GEN_NM,GEN_ID,GEN_NM,CBP_GEN_ID,CAPACITY,PATN_DT,HR_01,HR_02,HR_03,...,HR_15,HR_16,HR_17,HR_18,HR_19,HR_20,HR_21,HR_22,HR_23,HR_24
0,(주)엘시스,elsys0001,236,쏠3호태양광발전소,AE98,90.0,2020-01-01,0.0,0.0,0.0,...,56.0,35.6,10.3,0.4,0.0,0.0,0.0,0.0,0.0,0.0
1,(주)엘시스,elsys0001,236,쏠3호태양광발전소,AE98,90.0,2020-01-02,0.0,0.0,0.0,...,50.2,31.4,9.2,0.5,0.0,0.0,0.0,0.0,0.0,0.0
2,(주)엘시스,elsys0001,236,쏠3호태양광발전소,AE98,90.0,2020-01-03,0.0,0.0,0.0,...,51.0,32.3,9.3,0.4,0.0,0.0,0.0,0.0,0.0,0.0
3,(주)엘시스,elsys0001,236,쏠3호태양광발전소,AE98,90.0,2020-01-04,0.0,0.0,0.0,...,50.6,30.7,10.1,0.4,0.0,0.0,0.0,0.0,0.0,0.0
4,(주)엘시스,elsys0001,236,쏠3호태양광발전소,AE98,90.0,2020-01-05,0.0,0.0,0.0,...,39.0,38.2,11.1,0.3,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510,(주)엘시스,elsys0001,236,쏠3호태양광발전소,AE98,90.0,2021-05-27,0.0,0.0,0.0,...,29.0,51.6,42.2,22.9,5.7,0.4,0.0,0.0,0.0,0.0
511,(주)엘시스,elsys0001,236,쏠3호태양광발전소,AE98,90.0,2021-05-28,0.0,0.0,0.0,...,10.3,42.8,38.0,20.8,5.5,0.4,0.0,0.0,0.0,0.0
512,(주)엘시스,elsys0001,236,쏠3호태양광발전소,AE98,90.0,2021-05-29,0.0,0.0,0.0,...,53.3,51.4,40.7,21.5,5.9,0.4,0.0,0.0,0.0,0.0
513,(주)엘시스,elsys0001,236,쏠3호태양광발전소,AE98,90.0,2021-05-30,0.0,0.0,0.0,...,64.2,54.5,40.4,21.9,5.5,0.4,0.0,0.0,0.0,0.0


90.0


,DATE,HR_01,HR_02,HR_03,HR_04,HR_05,HR_06,HR_07,HR_08,HR_09,...,HR_15,HR_16,HR_17,HR_18,HR_19,HR_20,HR_21,HR_22,HR_23,HR_24
392,20210129,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.6,...,66.5,52.0,27.9,4.3,0.0,0.0,0.0,0.0,0.0,0.0
393,20210130,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,7.6,...,46.2,34.9,17.4,2.9,0.0,0.0,0.0,0.0,0.0,0.0
394,20210131,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,8.9,...,26.8,12.6,5.3,0.6,0.0,0.0,0.0,0.0,0.0,0.0
395,20210201,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,...,4.9,4.2,2.2,0.7,0.0,0.0,0.0,0.0,0.0,0.0
396,20210202,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,10.1,...,69.8,57.3,34.7,6.5,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510,20210527,0.0,0.0,0.0,0.0,0.0,0.2,2.9,7.7,4.7,...,29.0,51.6,42.2,22.9,5.7,0.4,0.0,0.0,0.0,0.0
511,20210528,0.0,0.0,0.0,0.0,0.0,0.5,4.0,16.3,33.9,...,10.3,42.8,38.0,20.8,5.5,0.4,0.0,0.0,0.0,0.0
512,20210529,0.0,0.0,0.0,0.0,0.0,0.7,4.0,16.2,34.5,...,53.3,51.4,40.7,21.5,5.9,0.4,0.0,0.0,0.0,0.0
513,20210530,0.0,0.0,0.0,0.0,0.0,0.9,5.3,15.9,33.1,...,64.2,54.5,40.4,21.9,5.5,0.4,0.0,0.0,0.0,0.0


In [4]:
onm_col_list = onm_df.columns
onm_col_list = onm_col_list[1:]
for col in onm_col_list:
    print(col[-2:]+'\',\'', end='')

01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','

In [5]:
onm_df2 = pd.DataFrame(columns=['DATE','POWER'])
print(onm_df.shape)

for r in range(onm_df.shape[0]):
    #print(onm_df.iloc[r,0]+' ', end='')
    for c in range(1,onm_df.shape[1]):
        IDX = r*len(onm_col_list)+c-1
        TIME = onm_df.iloc[r,0]+onm_col_list[c-1][-2:]
        POWER = onm_df.iloc[r,c]
        onm_df2.loc[IDX] = [TIME, POWER]
        
display(onm_df2)      
onm_df2.to_csv(base_dir+"/_"+RSRSID_list[2]+"onm_df_"+save_name+"_1hour.csv",mode='w',index=False, encoding='CP949')

(123, 25)


,DATE,POWER
0,2021012901,0.0
1,2021012902,0.0
2,2021012903,0.0
3,2021012904,0.0
4,2021012905,0.0
...,...,...
2947,2021053120,0.8
2948,2021053121,0.0
2949,2021053122,0.0
2950,2021053123,0.0


# --------------------------------------------------------- 여기 위로 정상 -----------------------

In [ ]:
# 결측값 보간, reshape
pow_dataset = pow_dataset.interpolate(method='linear')
pow_dataset = pow_dataset.values.reshape(-1,1)
pow_dataset = pd.DataFrame(pow_dataset)
#pow_dataset.columns = ['pow']
#pow_dataset.to_csv(save_path+"/pow_"+save_name+".csv",mode='w',index=False)


# scale
sc_pow = MinMaxScaler(feature_range = (0, 1))
scaled_pow = sc_pow.fit_transform(pow_dataset.values)
df_pow = pd.DataFrame(scaled_pow, columns=pow_dataset.columns, index=list(pow_dataset.index.values))

In [ ]:

def get_SENSOR():

    # pow 파일 load
    dir_path = base_dir+'+SENSOR/csv/'
    file_list   = os.listdir(dir_path)
    print(len(file_list))
    hrPow  = []    

    # pow측정값 에러가 큰 일자 제거
    for filename in file_list:
        if (filename[:-4] not in err_date_list):
            if ((filename[:-4]>=date_start) & (filename<date_end)):
                newdata = pd.read_csv(dir_path+'/'+filename).values[:,0]
                sensor_df = pd.concat(sensor_df, newdata)#hrPow.append(filedata)
                #print(filename[:-4])

    
    #낮시간 추출 (5~20시)
    pow_dataset = pd.DataFrame(hrPow)
    pow_dataset[23] = 0# 23시 data 쌓이지 않으므로 0으로 채움
    pow_dataset =pow_dataset.iloc[:,powhr_start:powhr_end+1]
    pow_dataset.to_csv(save_path+"/pow_test_"+save_name+".csv",mode='w',index=False)

    # 결측값 보간, reshape
    pow_dataset = pow_dataset.interpolate(method='linear')
    pow_dataset = pow_dataset.values.reshape(-1,1)
    pow_dataset = pd.DataFrame(pow_dataset)
    pow_dataset.columns = ['pow']
    pow_dataset.to_csv(save_path+"/pow_"+save_name+".csv",mode='w',index=False)

    
    # scale
    sc_pow = MinMaxScaler(feature_range = (0, 1))
    scaled_pow = sc_pow.fit_transform(pow_dataset.values)
    df_pow = pd.DataFrame(scaled_pow, columns=pow_dataset.columns, index=list(pow_dataset.index.values))
    
    return df_pow, sc_pow

# SENSOR (timestep : 1hour, 10mins)

In [127]:
from datetime import datetime

# load csv data
filename_s = 'data/sensor_'
excel_df_s = pd.read_excel(filename_s+'20210128_20210510.xlsx', converters={'id':str, 'local_time':str}, usecols=None, index_col=None)
sensor_df = excel_df_s.copy()
sensor_df = sensor_df.drop(index=0, axis=0)
sensor_df.sort_values('id')
display(sensor_df.head(2))

#일시 패턴 변환(2019-08-20 5:00 -> 2019082005)
sensor_df['date_temp'] = sensor_df['local_time'].str[0:4]+sensor_df['local_time'].str[5:7]+sensor_df['local_time'].str[8:10]+sensor_df['local_time'].str[11:13]+sensor_df['local_time'].str[-5:-4].replace(" ","0")+sensor_df['local_time'].str[-4:-3]
sensor_df = sensor_df[(sensor_df['date_temp'].astype(str).str[0:10]>=date_start) & (sensor_df['date_temp'].astype(str).str[0:10]<date_end)]
#sensor_df = sensor_df.interpolate(method='linear')# 결측값 보간
sensor_df.insert(0, 'date', sensor_df['date_temp'].values, True)

# drop columns
sensor_df.drop(['loc_oid','api_server_time','local_time','id','date_temp'], axis=1, inplace=True)
sensor_df.to_csv(filename_s+save_name+"_preprocess_10mins.csv",mode='w',index=False)
display(sensor_df)

step = 6
hourlysensor = []
sensor_df['date'] = sensor_df['date'].astype('float')
for i in range(0, sensor_df.shape[0], step):
    hourlysensor.append(np.max(sensor_df[i:i+step]))
    #print('{}:{}\t'.format(i, i+step), end='')
hourlysensor_df = pd.DataFrame(hourlysensor,columns=sensor_df.columns)
hourlysensor_df['date'] = hourlysensor_df['date'].astype('str').str[0:10]
hourlysensor_df.sort_values('date')

hourlysensor_df.to_csv(filename_s+save_name+"_preprocess_1hour_max.csv",mode='w',index=False)
display(hourlysensor_df)    

,id,loc_oid,temp,humidity,windspeed,windgust,maxdailygust,winddir,hourlyrainin,dailyrainin,weeklyrainin,monthlyrainin,yearlyrainin,solarradiation,uv,feelslike,dewpoint,outside_status,api_server_time,local_time
1,14693,1,22.11,36,0,0,7.77,118,0,0,0,92.2,498.09,459.05,3,21.31,6.33,200,2021-05-10 14:26:00,2021-05-10 14:30:51
2,14692,1,22.27,35,0,0,7.77,200,0,0,0,92.2,498.09,363.58,3,21.47,6.07,200,2021-05-10 14:16:00,2021-05-10 14:20:51


,date,temp,humidity,windspeed,windgust,maxdailygust,winddir,hourlyrainin,dailyrainin,weeklyrainin,monthlyrainin,yearlyrainin,solarradiation,uv,feelslike,dewpoint,outside_status
1529,202104292350,11.61,84,1.6,3.3,10.01,257,0,2.28,2.28,99.82,393.19,0,0,11.61,9,200
1530,202104292340,12.27,83,2.19,4.38,10.01,188,0,2.28,2.28,99.82,393.19,0,0,12.27,9.47,200
1531,202104292330,13.38,81,1.6,2.19,10.01,266,7.11,2.28,2.28,99.82,393.19,0,0,13.38,10.19,200
1532,202104292320,14.5,79,1.11,2.19,10.01,231,7.11,1.27,1.27,98.8,392.17,0,0,14.5,10.9,200
1533,202104292310,14.72,62,0,1.11,10.01,81,0,0,0,97.53,390.9,0,0,14.72,7.51,200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14628,202101290040,-4.77,73,2.01,3.3,10.01,318,0,0,12.19,12.19,12.19,0,0,-8.03,-8.86,200
14629,202101290030,-4.77,74,2.99,4.38,10.01,10,0,0,12.19,12.19,12.19,0,0,-9.23,-8.68,200
14630,202101290020,-4.77,73,3.97,7.77,10.01,301,0,0,12.19,12.19,12.19,0,0,-10.13,-8.86,200
14631,202101290010,-4.72,77,1.6,3.3,10.01,0,0,0,12.19,12.19,12.19,0,0,-7.33,-8.12,200


,date,temp,humidity,windspeed,windgust,maxdailygust,winddir,hourlyrainin,dailyrainin,weeklyrainin,monthlyrainin,yearlyrainin,solarradiation,uv,feelslike,dewpoint,outside_status
0,2021042923,14.72,84,2.19,4.38,10.01,306,7.11,2.28,2.28,99.82,393.19,0.00,0,14.72,10.90,200
1,2021042922,14.50,76,1.11,3.30,10.01,309,0.00,0.00,0.00,97.53,390.90,0.00,0,14.50,10.21,200
2,2021042921,14.38,76,1.29,2.19,10.01,294,0.00,0.00,0.00,97.53,390.90,0.00,0,14.38,10.21,200
3,2021042920,14.27,76,0.00,0.00,10.01,222,0.00,0.00,0.00,97.53,390.90,0.00,0,14.27,9.46,200
4,2021042919,17.11,72,0.00,0.00,10.01,305,0.00,0.00,0.00,97.53,390.90,13.92,0,17.11,11.59,200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2179,2021012904,-5.72,77,2.99,5.49,11.08,351,0.00,0.00,12.19,12.19,12.19,0.00,0,-9.19,-9.10,200
2180,2021012903,-5.50,78,4.11,5.49,11.08,357,0.00,0.00,12.19,12.19,12.19,0.00,0,-5.50,-8.71,200
2181,2021012902,-5.27,79,4.38,8.89,11.08,358,0.00,0.00,12.19,12.19,12.19,0.00,0,-5.27,-8.44,200
2182,2021012901,-4.77,85,4.60,6.70,11.08,353,0.00,0.00,12.19,12.19,12.19,0.00,0,-7.91,-7.39,200


# SENSOR (10분 간격)

In [86]:
from datetime import datetime

# pow 파일 load
# load csv data
filename_s = 'data/sensor_20210128_20210510'

excel_df_s = pd.read_excel(filename_s+'.xlsx', converters={'id':str, 'local_time':str}, usecols=None, index_col=None)
sensor_df = excel_df_s.copy()
sensor_df = sensor_df.drop(index=0, axis=0)
sensor_df.sort_values('id')
display(sensor_df.head(2))
print(sensor_df.dtypes)

#sensor_df['local_time'] = pd.to_datetime(sensor_df['local_time'], format='%Y-%m-%d %H:%M:%S', errors='raise')
#sensor_df['dateee'] = sensor_df['local_time'].dt.year.astype(str)+sensor_df['local_time'].dt.month.astype(str)+sensor_df['local_time'].dt.month_name().astype(str)

#일시 패턴 변환(2019-08-20 5:00 -> 2019082005)
sensor_df['date_temp'] = sensor_df['local_time'].str[0:4]+sensor_df['local_time'].str[5:7]+sensor_df['local_time'].str[8:10]+sensor_df['local_time'].str[11:13]+sensor_df['local_time'].str[-5:-4].replace(" ","0")+sensor_df['local_time'].str[-4:-3]
sensor_df = sensor_df[(sensor_df['date_temp'].astype(str).str[0:10]>=date_start) & (sensor_df['date_temp'].astype(str).str[0:10]<date_end)]
#sensor_df = sensor_df.interpolate(method='linear')# 결측값 보간
sensor_df.insert(0, 'date', sensor_df['date_temp'].values, True)

# drop columns
sensor_df.drop(['loc_oid','api_server_time','local_time','id','date_temp'], axis=1, inplace=True)
display(sensor_df.head(2))
print(sensor_df.columns)

display(sensor_df)
sensor_df.to_csv(filename_s+"_preprocess2.csv",mode='w',index=False)

,id,loc_oid,temp,humidity,windspeed,windgust,maxdailygust,winddir,hourlyrainin,dailyrainin,weeklyrainin,monthlyrainin,yearlyrainin,solarradiation,uv,feelslike,dewpoint,outside_status,api_server_time,local_time
1,14693,1,22.11,36,0,0,7.77,118,0,0,0,92.2,498.09,459.05,3,21.31,6.33,200,2021-05-10 14:26:00,2021-05-10 14:30:51
2,14692,1,22.27,35,0,0,7.77,200,0,0,0,92.2,498.09,363.58,3,21.47,6.07,200,2021-05-10 14:16:00,2021-05-10 14:20:51


id                 object
loc_oid            object
temp               object
humidity           object
windspeed          object
windgust           object
maxdailygust       object
winddir            object
hourlyrainin       object
dailyrainin        object
weeklyrainin       object
monthlyrainin      object
yearlyrainin       object
solarradiation     object
uv                 object
feelslike          object
dewpoint           object
outside_status     object
api_server_time    object
local_time         object
dtype: object


,date,temp,humidity,windspeed,windgust,maxdailygust,winddir,hourlyrainin,dailyrainin,weeklyrainin,monthlyrainin,yearlyrainin,solarradiation,uv,feelslike,dewpoint,outside_status
1529,202104292350,11.61,84,1.6,3.3,10.01,257,0,2.28,2.28,99.82,393.19,0,0,11.61,9,200
1530,202104292340,12.27,83,2.19,4.38,10.01,188,0,2.28,2.28,99.82,393.19,0,0,12.27,9.47,200


Index(['date', 'temp', 'humidity', 'windspeed', 'windgust', 'maxdailygust',
       'winddir', 'hourlyrainin', 'dailyrainin', 'weeklyrainin',
       'monthlyrainin', 'yearlyrainin', 'solarradiation', 'uv', 'feelslike',
       'dewpoint', 'outside_status'],
      dtype='object')


,date,temp,humidity,windspeed,windgust,maxdailygust,winddir,hourlyrainin,dailyrainin,weeklyrainin,monthlyrainin,yearlyrainin,solarradiation,uv,feelslike,dewpoint,outside_status
1529,202104292350,11.61,84,1.6,3.3,10.01,257,0,2.28,2.28,99.82,393.19,0,0,11.61,9,200
1530,202104292340,12.27,83,2.19,4.38,10.01,188,0,2.28,2.28,99.82,393.19,0,0,12.27,9.47,200
1531,202104292330,13.38,81,1.6,2.19,10.01,266,7.11,2.28,2.28,99.82,393.19,0,0,13.38,10.19,200
1532,202104292320,14.5,79,1.11,2.19,10.01,231,7.11,1.27,1.27,98.8,392.17,0,0,14.5,10.9,200
1533,202104292310,14.72,62,0,1.11,10.01,81,0,0,0,97.53,390.9,0,0,14.72,7.51,200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14628,202101290040,-4.77,73,2.01,3.3,10.01,318,0,0,12.19,12.19,12.19,0,0,-8.03,-8.86,200
14629,202101290030,-4.77,74,2.99,4.38,10.01,10,0,0,12.19,12.19,12.19,0,0,-9.23,-8.68,200
14630,202101290020,-4.77,73,3.97,7.77,10.01,301,0,0,12.19,12.19,12.19,0,0,-10.13,-8.86,200
14631,202101290010,-4.72,77,1.6,3.3,10.01,0,0,0,12.19,12.19,12.19,0,0,-7.33,-8.12,200


# weather (ASOS)

In [108]:
# pow 파일 load
# load csv data
filename_w = 'data/ASOS_'
weather_origin = read_csv(filename_w+'20200428_20210429.csv', encoding='CP949')
weather_df = weather_origin.copy()
weather_df.drop(['지점','지점명'], axis=1, inplace=True)
weather_df.drop(['기온 QC플래그','강수량 QC플래그','풍속 QC플래그','풍향 QC플래그','습도 QC플래그'], axis=1, inplace=True)
weather_df.drop(['현지기압 QC플래그','해면기압 QC플래그','일조 QC플래그','지면온도 QC플래그'], axis=1, inplace=True)
weather_df.drop(['5cm 지중온도(°C)','10cm 지중온도(°C)','20cm 지중온도(°C)','30cm 지중온도(°C)'], axis=1, inplace=True)
weather_df.drop(['3시간신적설(cm)','일사(MJ/m2)','운형(운형약어)','지면상태(지면상태코드)','현상번호(국내식)'], axis=1, inplace=True)

# set column name
weather_df.columns = ['date', 'temprt', 'rain', 'wnd_spd', 'wnd_dir', 'humdt','steampressr',
                   'dewpnt', 'pressr','seapressr','sunshine','snow','cloud','cloud2','mincloud','visiblt','grd_temprt']

# prioirty sort (피어슨상관계수)
weather_df = weather_df[['date','sunshine','humdt','wnd_spd','visiblt','cloud2',
                   'cloud','grd_temprt','wnd_dir','dewpnt','steampressr','temprt',
                   'mincloud','rain','pressr','seapressr','snow']]


# set NA data (관측값 0이 누적되어 결측된 경우. 0으로 세팅)
weather_df['rain'].fillna(0, inplace=True)     #강수량
weather_df['sunshine'].fillna(0, inplace=True) #일조
weather_df['snow'].fillna(0, inplace=True)     #적설량

#일시 패턴 변환(2019-08-20 5:00 -> 2019082005)
weather_df['date'] = weather_df['date'].str[0:4]+weather_df['date'].str[5:7]+weather_df['date'].str[8:10]+weather_df['date'].str[-5:-4].replace(" ","0")+weather_df['date'].str[-4:-3]
weather_df = weather_df[(weather_df['date'].astype(str).str[0:10]>=date_start) & (weather_df['date'].astype(str).str[0:10]<date_end)]
weather_df = weather_df.interpolate(method='linear')# 결측값 보간

# save file (test용)
weather_df = weather_df.sort_values('date')
weather_df.to_csv(filename_w+save_name+"_preprocess.csv",mode='w',index=False)
display(weather_df)
print(weather_df.shape)

,date,sunshine,humdt,wnd_spd,visiblt,cloud2,cloud,grd_temprt,wnd_dir,dewpnt,steampressr,temprt,mincloud,rain,pressr,seapressr,snow
2183,2021012900,0.0,89.0,5.2,110.0,3.0,8.0,0.6,290.0,-6.9,3.6,-5.4,1.000000,0.0,1001.1,1022.3,0.0
2182,2021012901,0.0,80.0,5.2,321.0,7.0,8.0,0.6,320.0,-8.4,3.2,-5.5,1.000000,0.0,1001.1,1022.3,0.0
2181,2021012902,0.0,78.0,6.8,548.0,2.0,2.0,0.6,320.0,-9.1,3.1,-5.9,1.000000,0.0,1001.4,1022.6,0.0
2180,2021012903,0.0,75.0,5.9,923.0,3.0,3.0,0.5,290.0,-10.0,2.9,-6.3,2.000000,0.0,1001.6,1022.9,0.0
2179,2021012904,0.0,76.0,6.6,562.0,0.0,0.0,0.4,290.0,-10.1,2.8,-6.6,2.000000,0.0,1001.7,1023.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,2021042919,0.3,43.0,1.6,1663.0,0.0,0.0,17.4,270.0,4.6,8.5,17.4,34.285714,0.0,984.0,1003.0,0.0
3,2021042920,0.0,58.0,0.7,1596.0,1.0,6.0,15.1,270.0,6.3,9.5,14.5,36.000000,0.0,984.6,1003.8,0.0
2,2021042921,0.0,66.0,1.8,1337.0,5.0,8.0,14.3,110.0,8.3,10.9,14.6,36.000000,0.0,984.3,1003.6,0.0
1,2021042922,0.0,58.0,1.7,1333.0,8.0,10.0,14.2,200.0,7.0,10.0,15.3,30.000000,0.0,985.2,1004.5,0.0


(2184, 17)


# power (sensor용, 10분단위 복제)

In [18]:
power_df = read_csv('./data/'+'power_20210129_20210429_preprocess_interpolate.csv', encoding='CP949')
print(power_df.shape)

power_df = read_csv('./data/'+'power_df.csv', encoding='CP949')
print(power_df.shape)

power_df = pd.read_excel('./data/power_20210129_20210429_preprocess.xlsx', usecols=None, index_col=None)
print(power_df.shape)

power_df = read_csv('./data/'+'power_20210129_20210429_preprocess_10mins.csv', encoding='CP949')
print(power_df.shape)
display(power_df)

power_df = read_csv('./data/'+'sensor_20210129_20210429_preprocess_10mins.csv', encoding='CP949')
print(power_df.shape)
display(power_df)

(2184, 2)
(2182, 2)
(2182, 2)
(13092, 2)


,date,power
0,202101290000,0.0
1,202101290010,0.0
2,202101290020,0.0
3,202101290030,0.0
4,202101290040,0.0
...,...,...
13087,202104292310,0.0
13088,202104292320,0.0
13089,202104292330,0.0
13090,202104292340,0.0


(13104, 17)


,date,temp,humidity,windspeed,windgust,maxdailygust,winddir,hourlyrainin,dailyrainin,weeklyrainin,monthlyrainin,yearlyrainin,solarradiation,uv,feelslike,dewpoint,outside_status
0,202104292350,11.61,84,1.60,3.30,10.01,257,0.00,2.28,2.28,99.82,393.19,0.0,0,11.61,9.00,200
1,202104292340,12.27,83,2.19,4.38,10.01,188,0.00,2.28,2.28,99.82,393.19,0.0,0,12.27,9.47,200
2,202104292330,13.38,81,1.60,2.19,10.01,266,7.11,2.28,2.28,99.82,393.19,0.0,0,13.38,10.19,200
3,202104292320,14.50,79,1.11,2.19,10.01,231,7.11,1.27,1.27,98.80,392.17,0.0,0,14.50,10.90,200
4,202104292310,14.72,62,0.00,1.11,10.01,81,0.00,0.00,0.00,97.53,390.90,0.0,0,14.72,7.51,200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13099,202101290040,-4.77,73,2.01,3.30,10.01,318,0.00,0.00,12.19,12.19,12.19,0.0,0,-8.03,-8.86,200
13100,202101290030,-4.77,74,2.99,4.38,10.01,10,0.00,0.00,12.19,12.19,12.19,0.0,0,-9.23,-8.68,200
13101,202101290020,-4.77,73,3.97,7.77,10.01,301,0.00,0.00,12.19,12.19,12.19,0.0,0,-10.13,-8.86,200
13102,202101290010,-4.72,77,1.60,3.30,10.01,0,0.00,0.00,12.19,12.19,12.19,0.0,0,-7.33,-8.12,200


In [21]:
'''
power_file  = './data/power_20200428_20210429_preprocess2.xlsx'
excelFile   = openpyxl.load_workbook(power_file)
sheets      = excelFile.get_sheet_names()
power_df = pd.read_excel(power_file, sheet_name=0, usecols=None, index_col=None, converters={'date':int})
'''
filename_p = './data/power_'

power_df = read_csv(filename_p+'20210129_20210429_preprocess_interpolate.csv', encoding='CP949')
print(power_df.shape)

power_df = power_df[(power_df['date'].astype(str).str[0:8]>=date_start) & (power_df['date'].astype(str).str[0:8]<date_end)]
power_df = power_df.sort_values('date')

power_df['date'] = power_df['date'].astype(int).astype(str)
power_df.to_csv('./data/power_20210129_20210429_preprocess_1hour.csv',mode='w',index=False)
display(power_df.head(1))

power_temp = power_df.copy()
power_temp['power'] = np.nan
for i in range (1,6):
    new_date = power_temp.copy()
    new_date['date'] = power_temp['date'].astype(str)+str(i)+'0'
    display(new_date)#.head(1))
    power_df = power_df.append(new_date)

    #isplay(power_df.head(1))

power_df['date'] = power_df['date'].astype(str).str.ljust(12,'0').astype(str)
power_df = power_df.sort_values('date')
display(power_df)
#power_df.drop(['date'], axis=1, inplace=True)

# interpolate
power_df = power_df.interpolate(method='linear')

power_df.to_csv(filename_p+save_name+"_preprocess_10mins.csv",mode='w',index=False)

(2184, 2)


,date,power
0,2021012900,0.0


,date,power
0,202101290010,NaN
1,202101290110,NaN
2,202101290210,NaN
3,202101290310,NaN
4,202101290410,NaN
...,...,...
2179,202104291910,NaN
2180,202104292010,NaN
2181,202104292110,NaN
2182,202104292210,NaN


,date,power
0,202101290020,NaN
1,202101290120,NaN
2,202101290220,NaN
3,202101290320,NaN
4,202101290420,NaN
...,...,...
2179,202104291920,NaN
2180,202104292020,NaN
2181,202104292120,NaN
2182,202104292220,NaN


,date,power
0,202101290030,NaN
1,202101290130,NaN
2,202101290230,NaN
3,202101290330,NaN
4,202101290430,NaN
...,...,...
2179,202104291930,NaN
2180,202104292030,NaN
2181,202104292130,NaN
2182,202104292230,NaN


,date,power
0,202101290040,NaN
1,202101290140,NaN
2,202101290240,NaN
3,202101290340,NaN
4,202101290440,NaN
...,...,...
2179,202104291940,NaN
2180,202104292040,NaN
2181,202104292140,NaN
2182,202104292240,NaN


,date,power
0,202101290050,NaN
1,202101290150,NaN
2,202101290250,NaN
3,202101290350,NaN
4,202101290450,NaN
...,...,...
2179,202104291950,NaN
2180,202104292050,NaN
2181,202104292150,NaN
2182,202104292250,NaN


,date,power
0,202101290000,0.0
0,202101290010,NaN
0,202101290020,NaN
0,202101290030,NaN
0,202101290040,NaN
...,...,...
2183,202104292310,NaN
2183,202104292320,NaN
2183,202104292330,NaN
2183,202104292340,NaN


In [2]:
'''
power_file  = './data/power_20200428_20210429_preprocess2.xlsx'
excelFile   = openpyxl.load_workbook(power_file)
sheets      = excelFile.get_sheet_names()
power_df = pd.read_excel(power_file, sheet_name=0, usecols=None, index_col=None, converters={'date':int})
'''
filename_p = './data/power_'

power_df = read_csv(filename_p+'df.csv', encoding='CP949')
print(power_df.shape)

power_df = power_df[(power_df['date'].astype(str).str[0:8]>=date_start) & (power_df['date'].astype(str).str[0:8]<date_end)]
power_df = power_df.sort_values('date')

power_df['date'] = power_df['date'].astype(int).astype(str)
power_df.to_csv('./data/power_df.csv',mode='w',index=False)
display(power_df.head(1))

power_temp = power_df.copy()
for i in range (1,6):
    new_date = power_temp.copy()
    new_date['date'] = new_date['date'].astype(str)+str(i)+'0'
    display(new_date.head(1))
    power_df = power_df.append(new_date)

    #isplay(power_df.head(1))

power_df['date'] = power_df['date'].astype(str).str.ljust(12,'0').astype(str)
power_df = power_df.sort_values('date')
display(power_df)
#power_df.drop(['date'], axis=1, inplace=True)

# interpolate
power_df = power_df.interpolate(method='linear')

power_df.to_csv(filename_p+save_name+"_preprocess_10mins.csv",mode='w',index=False)

(2182, 2)


,date,power
0,2021012900,0.0


,date,power
0,202101290010,0.0


,date,power
0,202101290020,0.0


,date,power
0,202101290030,0.0


,date,power
0,202101290040,0.0


,date,power
0,202101290050,0.0


,date,power
0,202101290000,0.0
0,202101290010,0.0
0,202101290020,0.0
0,202101290030,0.0
0,202101290040,0.0
...,...,...
2181,202104292310,0.0
2181,202104292320,0.0
2181,202104292330,0.0
2181,202104292340,0.0


# power (default)

In [ ]:
#############################################
# 태양광 전력
#############################################
power_file  = './data/power_20200428_20210429_preprocess.xlsx'
excelFile   = openpyxl.load_workbook(power_file)
sheets      = excelFile.get_sheet_names()
power_df = pd.read_excel(power_file, sheet_name=0, usecols=None, index_col=None, converters={'date':int})
power_df = power_df[(power_df['date'].astype(str).str[0:8]>=date_start) & (power_df['date'].astype(str).str[0:8]<date_end)]
power_df = power_df.sort_values('date')

power_df['date'] = power_df['date'].astype(int).astype(str)
power_df.to_csv('./data/power_df.csv',mode='w',index=False)
display(power_df)

#power_df.drop(['date'], axis=1, inplace=True)

# interpolate
power_df = power_df.interpolate(method='linear')

#############################################
# 종관기상관측
#############################################
weaather_file = 'data/ASOS_20200428_20210429_preprocess'
weather_df = read_csv(weaather_file+'.csv', encoding='CP949', converters={'date':int})
weather_df = weather_df[(weather_df['date'].astype(str).str[0:8]>=date_start) & (weather_df['date'].astype(str).str[0:8]<date_end)]
weather_df = weather_df.sort_values('date')

weather_df['date'] = weather_df['date'].astype(int).astype(str)
#display(weather_df)

weather_df.to_csv('./data/weather_df.csv',mode='w',index=False)
# normalization
#weather_df.drop(['date'], axis=1, inplace=True)
#weather_df = weather_df.astype('float32')
weather_df = weather_df.interpolate(method='linear')
weather_df = weather_df.iloc[:, 0:data_dim] #feature size 조절

#############################################
# JOIN
#############################################
total_df = pd.merge(power_df,weather_df, how='outer',on='date')
total_df = total_df.interpolate(method='linear')# 결측값 보간
#total_df['power'].fillna(0, inplace=True)
total_df = total_df.sort_values('date')
total_df.to_csv('./data/total_df.csv',mode='w',index=False)
display(total_df)

total_df.iloc[:,0:2].to_csv('./data/power_20210129_20210429_preprocess_interpolate.csv',mode='w',index=False)

In [111]:
#filename    = './data/power_20200428_20210429'
filename_p = './data/power_'
excelFile   = openpyxl.load_workbook(filename_p+'20200428_20210429.xlsx')
sheets      = excelFile.get_sheet_names()
writer      = pd.ExcelWriter(filename_p+save_name+"_preprocess.xlsx")
infoSheet   = pd.DataFrame(columns = ['sheetNm','generatorID','generatorNm','marketID','marketNm','capacity','startDate','endDate'])
#nfoSheet.columns = ['sheetNm','generatorID','generatorNm','marketID','marketNm','capacity']
    
print(sheets)
for s in range (len(sheets)-1):
    
    # excel_df: 원본
    excel_df = pd.read_excel(filename+'.xlsx', sheet_name=s, converters={'소규모발전기ID':str}, usecols=None, index_col=None)
    power_df = excel_df.copy()
        
    # power_df: 전처리
    power_df = power_df.rename(columns={"소규모발전기ID": "generatorID"})
    power_df = power_df.rename(columns={"전력시장ID": "marketID"})
    power_df = power_df.rename(columns={"설비용량(kW)": "capacity"})
    power_df = power_df.rename(columns={"날짜": "date"})
    power_df = power_df.rename(columns={"계량시간": "hour"})
    power_df = power_df.rename(columns={"발전량": "power"})
    
    # 일시 패턴 변환(2019-08-20 5:00 -> 2019082005)
    power_df['date'] = power_df['date'].astype(str).str[0:4]+power_df['date'].astype(str).str[5:7]+power_df['date'].astype(str).str[8:10]+power_df['hour'].astype(str).str.zfill(2)
    power_df = power_df.sort_values('date')
    
    # save excel
    sheetNm = str(power_df['generatorID'][0])+'_'+str(power_df['capacity'][0])
    infoData = [sheetNm,power_df['소규모발전기명'][0],power_df['generatorID'][0],power_df['전력시장발전기명'][0],power_df['marketID'][0],power_df['capacity'][0],np.min(power_df['date']),np.max(power_df['date'])]
    infoSheet.loc[len(infoSheet)] = infoData
    power_df.drop(['번호', '중개사업자명', '집합전력자원명', '소규모발전기명', '전력시장발전기명'], axis=1, inplace=True)
    power_df.drop(['hour', 'capacity', 'marketID','generatorID'], axis=1, inplace=True)
    power_df = power_df[(power_df['date'].astype(str).str[0:10]>=date_start) & (power_df['date'].astype(str).str[0:10]<date_end)]
    power_df = power_df.sort_values('date')
    power_df.to_excel(writer, sheet_name=sheetNm, index=False)
    print(power_df.shape)
    print(infoData)
    

pd.DataFrame(infoSheet).to_excel(writer, sheet_name='Sheet_Info', index=False)
writer.save()

<ipython-input-111-d6b8567134ad>:4: DeprecationWarning: Call to deprecated function get_sheet_names (Use wb.sheetnames).
  sheets      = excelFile.get_sheet_names()


['낙안솔라1호', '쏠태양광발전소2호', '온스타16호', '쏠태양광발전소4호', 'Sheet1']
(8754, 2)
['00234_89.7', '쏠 태양광발전소', '00234', '쏠태양광발전소', 'AE96', 89.7, '2020042800', '2021042923']
(4916, 2)
['00235_96.6', '쏠2호 태양광발전소', '00235', '쏠2호 태양광발전소', 'AE97', 96.6, '2020042805', '2021042923']
(8753, 2)
['00236_90', '쏠3호 태양광발전소', '00236', '쏠3호태양광발전소', 'AE98', 90, '2020042800', '2021042923']
(4928, 2)
['00237_46.2', '쏠4호 태양광발전소', '00237', '쏠4호태양광발전소', 'AE99', 46.2, '2020042805', '2021042923']


In [6]:
excelFile   = openpyxl.load_workbook('./data/power_20200428_20210429_preprocess.xlsx')
sheets      = excelFile.get_sheet_names()
    
print(sheets)
for s in range (len(sheets)-1):
    excel_df = pd.read_excel(filename+'.xlsx', sheet_name=s, converters={'소규모발전기ID':str}, usecols=None, index_col=None)
# power_df 
power_df = power_df_origin.copy()
power_df.drop(['번호', '중개사업자명', '집합전력자원명', '소규모발전기명', '전력시장발전기명'], axis=1, inplace=True)

power_df = power_df.rename(columns={"소규모발전기ID": "generatorID"})
power_df = power_df.rename(columns={"전력시장ID": "marketID"})
power_df = power_df.rename(columns={"설비용량(kW)": "capacity"})
power_df = power_df.rename(columns={"날짜": "date"})
power_df = power_df.rename(columns={"계량시간": "hour"})
power_df = power_df.rename(columns={"발전량": "power"})
print(power_df.head(3))

power_df.to_csv(filename+'_eng.csv')

<ipython-input-6-408b41ca854e>:2: DeprecationWarning: Call to deprecated function get_sheet_names (Use wb.sheetnames).
  sheets      = excelFile.get_sheet_names()


['00234_89.7', '00235_96.6', '00236_90', '00237_46.2', 'Sheet_Info']


NameError: name 'power_df_origin' is not defined

In [20]:
#일시 패턴 변환(2019-08-20 5:00 -> 2019082005)
power_df['date'] = power_df['date'].astype(str).str[0:4]+power_df['date'].astype(str).str[5:7]+power_df['date'].astype(str).str[8:10]+power_df['hour'].astype(str).str.zfill(2)
power_df    = power_df.rename(columns={"date": "ymdhms"})
generatorID = power_df['generatorID'][0]
capacity    = power_df['capacity'][0]

power_df.drop(['generatorID','capacity','marketID','hour'], axis=1, inplace=True)
print("generatorID: {}, capacity: {}".format(generatorID, capacity))
print(power_df.head(3))

     generatorID marketID  capacity       date  hour  power
4519       00236     AE98        90 2021-04-29     0    0.0
4520       00236     AE98        90 2021-04-29     1    0.0
4521       00236     AE98        90 2021-04-29     2    0.0
generatorID: 00236, capacity: 90
          ymdhms  power
4519  2021042900    0.0
4520  2021042901    0.0
4521  2021042902    0.0


In [38]:
import os
import fnmatch
from pandas import read_csv

def getWeather():
# pow 파일 load
    for filename in os.listdir('./data/'):
        if fnmatch.fnmatch(filename, '*ASOS_*.csv'):
            print(filename)

            # load csv data
            weather_df_origin = read_csv('./data/'+filename, encoding='CP949')
            weather_df = weather_df_origin.copy()
            print(weather_df.columns)
            
            weather_df.drop(['지점','지점명'], axis=1, inplace=True)
            weather_df.drop(['기온 QC플래그','강수량 QC플래그','풍속 QC플래그','풍향 QC플래그','습도 QC플래그'], axis=1, inplace=True)
            weather_df.drop(['현지기압 QC플래그','해면기압 QC플래그','일조 QC플래그','지면온도 QC플래그'], axis=1, inplace=True)
            weather_df.drop(['5cm 지중온도(°C)','10cm 지중온도(°C)','20cm 지중온도(°C)','30cm 지중온도(°C)'], axis=1, inplace=True)
            weather_df.drop(['3시간신적설(cm)','일사(MJ/m2)','운형(운형약어)','지면상태(지면상태코드)','현상번호(국내식)'], axis=1, inplace=True)

            print(weather_df.columns)
            # set column name
            weather_df.columns = ['date','temperature','precipitation','windSpeed','windDirection','humidity','vaporPressure', 'dewPoint'
                                  ,'localPressure','seaSurfacePressure','sunshine','Snow','cloudVolume','middleLowerLayer Cloud volume (10th quartile)','Lowest cloud height (100m)','Visibility (10m)','Ground temperature']

            print(weather_df.columns)
            # prioirty sort (피어슨상관계수)
            dataset = dataset[['ymdhms','sunshine','humdt','wnd_spd','visiblt','cloud2',
                               'cloud','grd_temprt','wnd_dir','dewpnt','steampressr','temprt',
                               'mincloud','rain','pressr','seapressr','snow']]


            # set NA data (관측값 0이 누적되어 결측된 경우. 0으로 세팅)
            dataset['rain'].fillna(0, inplace=True)     #강수량
            dataset['sunshine'].fillna(0, inplace=True) #일조
            dataset['snow'].fillna(0, inplace=True)     #적설량

            #일시 패턴 변환(2019-08-20 5:00 -> 2019082005)
            dataset['ymdhms'] = dataset['ymdhms'].str[0:4]+dataset['ymdhms'].str[5:7]+dataset['ymdhms'].str[8:10]+dataset['ymdhms'].str[-5:-4].replace(" ","0")+dataset['ymdhms'].str[-4:-3]
            # pow측정값 중 결측값 많은 일자 제거
            dataset = dataset[(dataset['ymdhms'].str[0:8]>=date_start) & (dataset['ymdhms'].str[0:8]<date_end)]
            for err_date in err_date_list:
                idx_err = dataset[dataset['ymdhms'].str.startswith(err_date)].index
                dataset = dataset.drop(idx_err)

            #낮시간 추출 (5~20시)
            dataset = dataset[(dataset['ymdhms'].str[-2:]>=str(powhr_start).rjust(2, '0')) &(dataset['ymdhms'].str[-2:]<=str(powhr_end))]
            dataset = dataset.interpolate(method='linear')# 결측값 보간
            
            # save file (test용)
            dataset.to_csv(save_path+"/weather_"+save_name+".csv",mode='w',index=False)

            # normalization
            dataset.drop(['ymdhms'], axis=1, inplace=True)
            dataset = dataset.astype('float32')
            dataset = dataset.interpolate(method='linear')
            
            #YEO-JOHNSON transform
            #yeo_df = yeo_johnson_transform(dataset)
            yeo_df = dataset
            #insert feature (test)
            yeo_df.insert(2, 'temp_press', yeo_df['temprt']-yeo_df['steampressr'], True)
            yeo_df.insert(2, 'sunshine_humdt', abs(yeo_df['sunshine'])-(yeo_df['humdt']*(2.1)), True)#0.35
            
            sc = MinMaxScaler(feature_range = (0, 1))#scale
            scaled_weather = sc.fit_transform(yeo_df.values)
            weather = pd.DataFrame(scaled_weather, columns=yeo_df.columns, index=list(yeo_df.index.values))
            
            weather = weather.iloc[:, 0:data_dim] #feature size 조절
    return weather

print(getWeather())
    

ASOS_20200428_20210429.csv
Index(['지점', '지점명', '일시', '기온(°C)', '기온 QC플래그', '강수량(mm)', '강수량 QC플래그',
       '풍속(m/s)', '풍속 QC플래그', '풍향(16방위)', '풍향 QC플래그', '습도(%)', '습도 QC플래그',
       '증기압(hPa)', '이슬점온도(°C)', '현지기압(hPa)', '현지기압 QC플래그', '해면기압(hPa)',
       '해면기압 QC플래그', '일조(hr)', '일조 QC플래그', '일사(MJ/m2)', '적설(cm)', '3시간신적설(cm)',
       '전운량(10분위)', '중하층운량(10분위)', '운형(운형약어)', '최저운고(100m )', '시정(10m)',
       '지면상태(지면상태코드)', '현상번호(국내식)', '지면온도(°C)', '지면온도 QC플래그', '5cm 지중온도(°C)',
       '10cm 지중온도(°C)', '20cm 지중온도(°C)', '30cm 지중온도(°C)'],
      dtype='object')
Index(['일시', '기온(°C)', '강수량(mm)', '풍속(m/s)', '풍향(16방위)', '습도(%)', '증기압(hPa)',
       '이슬점온도(°C)', '현지기압(hPa)', '해면기압(hPa)', '일조(hr)', '적설(cm)', '전운량(10분위)',
       '중하층운량(10분위)', '최저운고(100m )', '시정(10m)', '지면온도(°C)'],
      dtype='object')
Index(['Date', 'Air temperature (°C)', 'Precipitation (mm)',
       'Wind speed (m/s)', 'Wind direction (16 directions)', 'Humidity (%)',
       'Vapor pressure (hPa)', 'Dew point temperature (°C)',
   

UnboundLocalError: local variable 'dataset' referenced before assignment

In [ ]:
print(power_df.columns)
power_df.columns = ['date', 'hour', 'power', 'YMDH', 'month', 'fullday', 'season', 'year',
        'Temperature', 'Precipitation', 'Wind speed', 'Wind direction', 'Humidity',
       'Vapor pressure','Dew point','Local pressure','Sea pressure','Sunlight',
        'Irradiance','Snowfall','Total cloud',
        'Visibility','Ground temperature']
print(testDF.columns)

incheon_wether = incheon_wether.rename(columns={"일시": "YMDH"})

In [66]:
wether = read_csv('./data/weather.csv', encoding='CP949')
incheon_wether = read_csv('./data/incheon_weather.csv',encoding='CP949')
#korea_pow = read_csv('./data/korea_pow.csv', encoding='CP949')
korea_pow = read_csv('./data/korea_pow.csv', encoding='CP949')

In [67]:
print(korea_dust)
#print(incheon_dust)
print(wether)

korea_pow["YMDH"] = korea_pow["거래일"] +korea_pow["시간"].astype(str).str.zfill(2)

       STATION_ID STATION_NAME        YMDH    SO2   CO     O3    NO2   PM10  \
0          111121       junggu  2015010101  0.006  0.6  0.022  0.011   44.0   
1          111121       junggu  2015010102  0.006  0.6  0.022  0.010   57.0   
2          111121       junggu  2015010103  0.006  0.6  0.021  0.010   76.0   
3          111121       junggu  2015010104  0.006  0.6  0.022  0.009   86.0   
4          111121       junggu  2015010105  0.006  0.5  0.021  0.009  106.0   
...           ...          ...         ...    ...  ...    ...    ...    ...   
51883      111121       junggu  2020103120  0.003  0.4  0.021  0.035   34.0   
51884      111121       junggu  2020103121  0.003  0.4  0.025  0.028   32.0   
51885      111121       junggu  2020103122  0.003  0.4  0.030  0.023   30.0   
51886      111121       junggu  2020103123  0.003  0.4  0.034  0.019   27.0   
51887      111121       junggu  2020103124  0.003  0.4  0.033  0.019   27.0   

       PM25  
0       7.0  
1      10.0  
2      10

In [68]:
korea_pow['YMDH'] = korea_pow['YMDH'].str.replace('-', '', regex=False)

In [69]:
korea_pow['month'] = korea_pow['YMDH'].str.slice(start=4, stop=6)
korea_pow['fullday'] = korea_pow['YMDH'].str.slice(start=0, stop=-2)
korea_pow['month'] = korea_pow['month'].astype('int')
korea_pow['season'] = "winter"
korea_pow['year'] = korea_pow['YMDH'].str.slice(start=0, stop=4)

korea_pow.loc[(korea_pow['month'] >=3) & (korea_pow['month'] <=5),'season'] = 'spring'
korea_pow.loc[(korea_pow['month'] >=6) & (korea_pow['month'] <=8),'season'] = 'summer'
korea_pow.loc[(korea_pow['month'] >=9) & (korea_pow['month'] <=11),'season'] = 'autumn'

In [70]:

wether['일시'] = wether['일시'].str.replace('-', '', regex=False)
wether['일시'] = wether['일시'].str.replace(':00', '', regex=False)
wether['일시'] = wether['일시'].str.replace(' ', '', regex=False)

incheon_wether['일시'] = incheon_wether['일시'].str.replace('-', '', regex=False)
incheon_wether['일시'] = incheon_wether['일시'].str.replace(':00', '', regex=False)
incheon_wether['일시'] = incheon_wether['일시'].str.replace(' ', '', regex=False)

In [71]:
wether = wether.rename(columns={"일시": "YMDH"})

incheon_wether = incheon_wether.rename(columns={"일시": "YMDH"})

incheon_dust = incheon_wether.rename(columns={"측정일시": "YMDH"})
kangil_dust = incheon_wether.rename(columns={"측정일시": "YMDH"})
kangdong_dust = incheon_wether.rename(columns={"측정일시": "YMDH"})


#부산 서울 제주 인천

In [72]:
wether

,지점,지점명,YMDH,기온(°C),강수량(mm),강수량 QC플래그,풍속(m/s),풍향(16방위),습도(%),증기압(hPa),이슬점온도(°C),현지기압(hPa),해면기압(hPa),일조(hr),일사(MJ/m2),적설(cm),3시간신적설(cm),전운량(10분위),중하층운량(10분위),운형(운형약어),최저운고(100m ),시정(10m),지면상태(지면상태코드),현상번호(국내식),지면온도(°C),5cm 지중온도(°C),10cm 지중온도(°C),20cm 지중온도(°C),30cm 지중온도(°C)
0,108,서울,2014010100,3.3,NaN,NaN,3.8,250.0,65.0,5.0,-2.6,1001.9,1012.5,NaN,NaN,NaN,NaN,6.0,6.0,Sc,10.0,600.0,NaN,40.0,0.0,0.1,-0.2,0.0,1.5
1,108,서울,2014010101,2.6,NaN,NaN,2.3,250.0,66.0,4.9,-3.1,1002.2,1012.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,-0.1,0.1,-0.2,0.1,1.5
2,108,서울,2014010102,1.7,NaN,NaN,1.7,250.0,67.0,4.6,-3.7,1002.4,1013.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,-0.3,0.0,-0.2,0.0,1.5
3,108,서울,2014010103,1.4,NaN,NaN,1.4,250.0,60.0,4.1,-5.5,1002.2,1012.9,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,600.0,4.0,4240.0,-0.4,0.0,-0.2,0.1,1.5
4,108,서울,2014010104,0.9,NaN,NaN,2.8,270.0,59.0,3.8,-6.2,1002.5,1013.2,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,600.0,NaN,42.0,-0.6,0.0,-0.2,0.0,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230775,184,제주,2020073119,27.4,NaN,NaN,1.5,20.0,86.0,31.3,24.8,1011.4,1013.8,0.0,0.29,NaN,NaN,NaN,9.0,NaN,67.0,1975.0,NaN,NaN,29.3,29.3,30.8,29.3,27.7
230776,184,제주,2020073120,26.6,NaN,NaN,0.6,270.0,91.0,31.7,25.0,1011.4,1013.8,0.0,0.03,NaN,NaN,7.0,7.0,NaN,67.0,1653.0,NaN,NaN,28.2,29.1,30.3,29.3,27.8
230777,184,제주,2020073121,26.3,NaN,NaN,1.2,20.0,92.0,31.3,24.8,1011.5,1013.9,NaN,NaN,NaN,NaN,7.0,0.0,Ci,NaN,1791.0,NaN,NaN,27.5,28.9,30.0,29.3,27.8
230778,184,제주,2020073122,26.9,NaN,NaN,1.3,50.0,85.0,30.0,24.1,1011.5,1013.9,NaN,NaN,NaN,NaN,4.0,0.0,NaN,NaN,2000.0,NaN,NaN,26.9,28.7,29.7,29.2,27.9


In [73]:
seoul_pow = korea_pow.loc[korea_pow['지역세부구분']=='서울시', :]
#korea_pow.loc[korea_pow['지역세부구분']=='강원도', :]
pusan_pow = korea_pow.loc[korea_pow['지역세부구분']=='경상남도', :]
jeju_pow = korea_pow.loc[korea_pow['지역세부구분']=='제주도', :]

In [74]:
seoul_weather = wether.loc[wether['지점명']=='서울', :]
pusan_weather = wether.loc[wether['지점명']=='부산', :]
jeju_weather = wether.loc[wether['지점명']=='제주', :]

In [75]:
jeju_weather.shape

(57694, 29)

In [76]:
seoul_dataset = seoul_pow.merge(seoul_weather, how='left', on='YMDH')
pusan_dataset = pusan_pow.merge(pusan_weather, how='left', on='YMDH')
jeju_dataset = jeju_pow.merge(jeju_weather, how='left', on='YMDH')

In [77]:
seoul_dataset

,연료원,지역세부구분,거래일,시간,전력거래량,YMDH,month,fullday,season,year,지점,지점명,기온(°C),강수량(mm),강수량 QC플래그,풍속(m/s),풍향(16방위),습도(%),증기압(hPa),이슬점온도(°C),현지기압(hPa),해면기압(hPa),일조(hr),일사(MJ/m2),적설(cm),3시간신적설(cm),전운량(10분위),중하층운량(10분위),운형(운형약어),최저운고(100m ),시정(10m),지면상태(지면상태코드),현상번호(국내식),지면온도(°C),5cm 지중온도(°C),10cm 지중온도(°C),20cm 지중온도(°C),30cm 지중온도(°C)
0,태양광,서울시,2014-01-01,1,0.000000,2014010101,1,20140101,winter,2014,108,서울,2.6,NaN,NaN,2.3,250.0,66.0,4.9,-3.1,1002.2,1012.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,-0.1,0.1,-0.2,0.1,1.5
1,태양광,서울시,2014-01-01,2,0.000000,2014010102,1,20140101,winter,2014,108,서울,1.7,NaN,NaN,1.7,250.0,67.0,4.6,-3.7,1002.4,1013.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,-0.3,0.0,-0.2,0.0,1.5
2,태양광,서울시,2014-01-01,3,0.000000,2014010103,1,20140101,winter,2014,108,서울,1.4,NaN,NaN,1.4,250.0,60.0,4.1,-5.5,1002.2,1012.9,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,600.0,4.0,4240.0,-0.4,0.0,-0.2,0.1,1.5
3,태양광,서울시,2014-01-01,4,0.000000,2014010104,1,20140101,winter,2014,108,서울,0.9,NaN,NaN,2.8,270.0,59.0,3.8,-6.2,1002.5,1013.2,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,600.0,NaN,42.0,-0.6,0.0,-0.2,0.0,1.5
4,태양광,서울시,2014-01-01,5,0.000000,2014010105,1,20140101,winter,2014,108,서울,0.6,NaN,NaN,2.0,270.0,56.0,3.6,-7.1,1002.1,1012.8,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,800.0,NaN,42.0,-0.7,0.0,-0.2,0.1,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57691,태양광,서울시,2020-07-31,20,0.061932,2020073120,7,20200731,summer,2020,108,서울,27.0,NaN,9.0,2.8,250.0,83.0,29.5,23.8,1002.8,1012.5,0.0,0.09,NaN,NaN,9.0,8.0,NaN,5.0,1909.0,NaN,NaN,25.7,27.6,26.9,26.2,25.5
57692,태양광,서울시,2020-07-31,21,0.000000,2020073121,7,20200731,summer,2020,108,서울,26.7,NaN,9.0,2.2,270.0,86.0,30.0,24.1,1003.0,1012.7,NaN,NaN,NaN,NaN,7.0,7.0,Sc,6.0,1519.0,NaN,NaN,25.3,27.3,26.8,26.2,25.5
57693,태양광,서울시,2020-07-31,22,0.000000,2020073122,7,20200731,summer,2020,108,서울,26.5,NaN,9.0,1.8,250.0,87.0,30.0,24.1,1003.1,1012.8,NaN,NaN,NaN,NaN,5.0,5.0,NaN,4.0,1381.0,NaN,NaN,25.4,27.1,26.6,26.1,25.6
57694,태양광,서울시,2020-07-31,23,0.000000,2020073123,7,20200731,summer,2020,108,서울,26.2,0.2,NaN,2.4,200.0,90.0,30.6,24.4,1003.1,1012.8,NaN,NaN,NaN,NaN,8.0,8.0,Sc,4.0,852.0,NaN,1901.0,25.3,26.9,26.4,26.1,25.6


In [78]:
seoul_weather

,지점,지점명,YMDH,기온(°C),강수량(mm),강수량 QC플래그,풍속(m/s),풍향(16방위),습도(%),증기압(hPa),이슬점온도(°C),현지기압(hPa),해면기압(hPa),일조(hr),일사(MJ/m2),적설(cm),3시간신적설(cm),전운량(10분위),중하층운량(10분위),운형(운형약어),최저운고(100m ),시정(10m),지면상태(지면상태코드),현상번호(국내식),지면온도(°C),5cm 지중온도(°C),10cm 지중온도(°C),20cm 지중온도(°C),30cm 지중온도(°C)
0,108,서울,2014010100,3.3,NaN,NaN,3.8,250.0,65.0,5.0,-2.6,1001.9,1012.5,NaN,NaN,NaN,NaN,6.0,6.0,Sc,10.0,600.0,NaN,40.0,0.0,0.1,-0.2,0.0,1.5
1,108,서울,2014010101,2.6,NaN,NaN,2.3,250.0,66.0,4.9,-3.1,1002.2,1012.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,-0.1,0.1,-0.2,0.1,1.5
2,108,서울,2014010102,1.7,NaN,NaN,1.7,250.0,67.0,4.6,-3.7,1002.4,1013.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,-0.3,0.0,-0.2,0.0,1.5
3,108,서울,2014010103,1.4,NaN,NaN,1.4,250.0,60.0,4.1,-5.5,1002.2,1012.9,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,600.0,4.0,4240.0,-0.4,0.0,-0.2,0.1,1.5
4,108,서울,2014010104,0.9,NaN,NaN,2.8,270.0,59.0,3.8,-6.2,1002.5,1013.2,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,600.0,NaN,42.0,-0.6,0.0,-0.2,0.0,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215439,108,서울,2020073119,27.4,NaN,9.0,1.8,230.0,82.0,29.8,24.0,1002.6,1012.3,0.0,0.15,NaN,NaN,9.0,9.0,NaN,5.0,1995.0,NaN,NaN,26.6,27.9,27.1,26.2,25.4
215440,108,서울,2020073120,27.0,NaN,9.0,2.8,250.0,83.0,29.5,23.8,1002.8,1012.5,0.0,0.09,NaN,NaN,9.0,8.0,NaN,5.0,1909.0,NaN,NaN,25.7,27.6,26.9,26.2,25.5
215441,108,서울,2020073121,26.7,NaN,9.0,2.2,270.0,86.0,30.0,24.1,1003.0,1012.7,NaN,NaN,NaN,NaN,7.0,7.0,Sc,6.0,1519.0,NaN,NaN,25.3,27.3,26.8,26.2,25.5
215442,108,서울,2020073122,26.5,NaN,9.0,1.8,250.0,87.0,30.0,24.1,1003.1,1012.8,NaN,NaN,NaN,NaN,5.0,5.0,NaN,4.0,1381.0,NaN,NaN,25.4,27.1,26.6,26.1,25.6


In [79]:
korea_dust['YMDH'] = korea_dust['YMDH'].astype(str)
korea_dust['fullday'] = korea_dust['YMDH'].str.slice(start=0, stop=-2)
korea_dust['month'] = korea_dust['YMDH'].str.slice(start=4, stop=6)

korea_dust['month'] = korea_dust['month'].astype('int')
korea_dust['season'] = "winter"
korea_dust['year'] = korea_dust['YMDH'].str.slice(start=0, stop=4)

korea_dust.loc[(korea_dust['month'] >=3) & (korea_dust['month'] <=5),'season'] = 'spring'
korea_dust.loc[(korea_dust['month'] >=6) & (korea_dust['month'] <=8),'season'] = 'summer'
korea_dust.loc[(korea_dust['month'] >=9) & (korea_dust['month'] <=11),'season'] = 'autumn'




#kangdong_dust = read_csv('./data/kangdong_dust.csv', encoding='CP949')

kangdong_dust['YMDH'] = kangdong_dust['YMDH'].astype(str)
kangdong_dust['fullday'] = kangdong_dust['YMDH'].str.slice(start=0, stop=-2)
kangdong_dust['month'] = kangdong_dust['YMDH'].str.slice(start=4, stop=6)

kangdong_dust['month'] = kangdong_dust['month'].astype('int')
kangdong_dust['season'] = "winter"
kangdong_dust['year'] = kangdong_dust['YMDH'].str.slice(start=0, stop=4)

kangdong_dust.loc[(kangdong_dust['month'] >=3) & (kangdong_dust['month'] <=5),'season'] = 'spring'
kangdong_dust.loc[(kangdong_dust['month'] >=6) & (kangdong_dust['month'] <=8),'season'] = 'summer'
kangdong_dust.loc[(kangdong_dust['month'] >=9) & (kangdong_dust['month'] <=11),'season'] = 'autumn'

#incheon_dust = read_csv('./data/incheon_dust.csv', encoding='CP949')

incheon_dust['YMDH'] = incheon_dust['YMDH'].astype(str)
incheon_dust['fullday'] = incheon_dust['YMDH'].str.slice(start=0, stop=-2)
incheon_dust['month'] = incheon_dust['YMDH'].str.slice(start=4, stop=6)

incheon_dust['month'] = incheon_dust['month'].astype('int')
incheon_dust['season'] = "winter"
incheon_dust['year'] = incheon_dust['YMDH'].str.slice(start=0, stop=4)

incheon_dust.loc[(incheon_dust['month'] >=3) & (incheon_dust['month'] <=5),'season'] = 'spring'
incheon_dust.loc[(incheon_dust['month'] >=6) & (incheon_dust['month'] <=8),'season'] = 'summer'
incheon_dust.loc[(incheon_dust['month'] >=9) & (incheon_dust['month'] <=11),'season'] = 'autumn'


#kangil_dust = read_csv('./data/kangil_dust.csv', encoding='CP949')

kangil_dust['YMDH'] = kangil_dust['YMDH'].astype(str)
kangil_dust['fullday'] = kangil_dust['YMDH'].str.slice(start=0, stop=-2)
kangil_dust['month'] = kangil_dust['YMDH'].str.slice(start=4, stop=6)

kangil_dust['month'] = kangil_dust['month'].astype('int')
kangil_dust['season'] = "winter"
kangil_dust['year'] = kangil_dust['YMDH'].str.slice(start=0, stop=4)

kangil_dust.loc[(kangil_dust['month'] >=3) & (kangil_dust['month'] <=5),'season'] = 'spring'
kangil_dust.loc[(kangil_dust['month'] >=6) & (kangil_dust['month'] <=8),'season'] = 'summer'
kangil_dust.loc[(kangil_dust['month'] >=9) & (kangil_dust['month'] <=11),'season'] = 'autumn'



In [80]:
kor_dust_dataset = korea_dust.merge(seoul_weather, how='left', on='YMDH')
kangil_dust_dataset = kangil_dust.merge(seoul_weather, how='left', on='YMDH')
kangdong_dust_dataset = kangdong_dust.merge(seoul_weather, how='left', on='YMDH')

incheon_dust_dataset = incheon_dust.merge(incheon_wether, how='left', on='YMDH')

In [82]:

seoul_dataset = seoul_dataset.fillna(0)
pusan_dataset = pusan_dataset.fillna(0)
jeju_dataset = jeju_dataset.fillna(0)



seoul_dataset.to_csv('./data/seoul_pow2.csv', encoding='CP949')
pusan_dataset.to_csv('./data/pusan_pow2.csv', encoding='CP949')
jeju_dataset.to_csv('./data/jeju_pow2.csv', encoding='CP949')

In [95]:
testDF = jeju_dataset.copy()
testDF.columns = jeju_dataset.columns
print(testDF.columns)

testDF.drop(['연료원', '지역세부구분','강수량 QC플래그','지면상태(지면상태코드)', '현상번호(국내식)', '운형(운형약어)','10cm 지중온도(°C)',
       '20cm 지중온도(°C)', '30cm 지중온도(°C)','3시간신적설(cm)','지점', '지점명','중하층운량(10분위)', '최저운고(100m )','5cm 지중온도(°C)'], axis=1, inplace=True)

print(testDF.columns)
testDF.columns = ['date', 'hour', 'power', 'YMDH', 'month', 'fullday', 'season', 'year',
        'Temperature', 'Precipitation', 'Wind speed', 'Wind direction', 'Humidity',
       'Vapor pressure','Dew point','Local pressure','Sea pressure','Sunlight',
        'Irradiance','Snowfall','Total cloud',
        'Visibility','Ground temperature']
print(testDF.columns)
testDF.to_csv('./data/jeju_pow3.csv', encoding='CP949')


Index(['연료원', '지역세부구분', '거래일', '시간', '전력거래량', 'YMDH', 'month', 'fullday',
       'season', 'year', '지점', '지점명', '기온(°C)', '강수량(mm)', '강수량 QC플래그',
       '풍속(m/s)', '풍향(16방위)', '습도(%)', '증기압(hPa)', '이슬점온도(°C)', '현지기압(hPa)',
       '해면기압(hPa)', '일조(hr)', '일사(MJ/m2)', '적설(cm)', '3시간신적설(cm)', '전운량(10분위)',
       '중하층운량(10분위)', '운형(운형약어)', '최저운고(100m )', '시정(10m)', '지면상태(지면상태코드)',
       '현상번호(국내식)', '지면온도(°C)', '5cm 지중온도(°C)', '10cm 지중온도(°C)',
       '20cm 지중온도(°C)', '30cm 지중온도(°C)'],
      dtype='object')
Index(['거래일', '시간', '전력거래량', 'YMDH', 'month', 'fullday', 'season', 'year',
       '기온(°C)', '강수량(mm)', '풍속(m/s)', '풍향(16방위)', '습도(%)', '증기압(hPa)',
       '이슬점온도(°C)', '현지기압(hPa)', '해면기압(hPa)', '일조(hr)', '일사(MJ/m2)', '적설(cm)',
       '전운량(10분위)', '시정(10m)', '지면온도(°C)'],
      dtype='object')
Index(['date', 'hour', 'power', 'YMDH', 'month', 'fullday', 'season', 'year',
       'Temperature', 'Precipitation', 'Wind speed', 'Wind direction',
       'Humidity', 'Vapor pressure', 'Dew point', '

In [96]:
jeju_dataset = read_csv('./data/jeju_pow3.csv')
jeju_dataset

,Unnamed: 0,date,hour,power,YMDH,month,fullday,season,year,Temperature,Precipitation,Wind speed,Wind direction,Humidity,Vapor pressure,Dew point,Local pressure,Sea pressure,Sunlight,Irradiance,Snowfall,Total cloud,Visibility,Ground temperature
0,0,2014-01-01,1,0.000000,2014010101,1,20140101,winter,2014,11.3,0.0,7.1,290.0,51.0,6.8,1.5,1013.8,1016.3,0.0,0.00,0.0,0.0,0.0,6.5
1,1,2014-01-01,2,0.000000,2014010102,1,20140101,winter,2014,10.7,0.0,7.0,290.0,51.0,6.5,0.9,1014.0,1016.5,0.0,0.00,0.0,0.0,0.0,6.5
2,2,2014-01-01,3,0.000000,2014010103,1,20140101,winter,2014,10.4,0.0,5.7,290.0,47.0,5.9,-0.4,1014.2,1016.7,0.0,0.00,0.0,0.0,1500.0,5.7
3,3,2014-01-01,4,0.000000,2014010104,1,20140101,winter,2014,10.5,0.0,6.4,290.0,47.0,6.0,-0.3,1014.0,1016.5,0.0,0.00,0.0,0.0,1200.0,5.5
4,4,2014-01-01,5,0.000000,2014010105,1,20140101,winter,2014,10.4,0.0,5.0,290.0,54.0,6.8,1.4,1013.5,1016.0,0.0,0.00,0.0,2.0,1200.0,5.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57691,57691,2020-07-31,20,10.137453,2020073120,7,20200731,summer,2020,26.6,0.0,0.6,270.0,91.0,31.7,25.0,1011.4,1013.8,0.0,0.03,0.0,7.0,1653.0,28.2
57692,57692,2020-07-31,21,7.378050,2020073121,7,20200731,summer,2020,26.3,0.0,1.2,20.0,92.0,31.3,24.8,1011.5,1013.9,0.0,0.00,0.0,7.0,1791.0,27.5
57693,57693,2020-07-31,22,2.683048,2020073122,7,20200731,summer,2020,26.9,0.0,1.3,50.0,85.0,30.0,24.1,1011.5,1013.9,0.0,0.00,0.0,4.0,2000.0,26.9
57694,57694,2020-07-31,23,0.376994,2020073123,7,20200731,summer,2020,27.3,0.0,0.3,0.0,79.0,28.6,23.3,1011.4,1013.8,0.0,0.00,0.0,6.0,2000.0,26.6


In [97]:
#seoul_dataset.to_csv('seoul_pow.csv')
#pusan_dataset.to_csv('pusan_pow.csv')
#jeju_dataset.to_csv('jeju_pow.csv')
#kor_dust_dataset.to_csv('kor_dust.csv')
pd.set_option('display.max_columns', 500)


In [98]:
def f(x):
    tmpdict = dict()
    columns = x.columns
    for CL in columns:
        #print(x[CL].dtype)
        if x[CL].dtype=='object':
            x[CL] = x[CL].apply(str)
            tmpdict[CL] = x[CL].max()
        else:
            tmpdict[CL] = x[CL].max()
    #print(tmpdict)
    
    return pd.Series(tmpdict)

seoul_dataset_day_df = seoul_dataset.groupby(by=["fullday"], dropna=False).apply(f)
pusan_dataset_day_df = pusan_dataset.groupby(by=["fullday"], dropna=False).apply(f)
jeju_dataset_day_df = jeju_dataset.groupby(by=["fullday"], dropna=False).apply(f)
kor_dust_dataset_day_df = kor_dust_dataset.groupby(by=["fullday"], dropna=False).apply(f)

kangil_dust_dataset_day_df = kangil_dust_dataset.groupby(by=["fullday"], dropna=False).apply(f)
kangdong_dust_dataset_day_df = kangdong_dust_dataset.groupby(by=["fullday"], dropna=False).apply(f)
incheon_dust_dataset_day_df = incheon_dust_dataset.groupby(by=["fullday"], dropna=False).apply(f)




In [ ]:
seoul_dataset_day_df

In [ ]:
seoul_dataset

In [102]:
seoul_dataset_day_df.to_csv('./data/seoul_pow_day2.csv')
pusan_dataset_day_df.to_csv('./data/pusan_pow_day2.csv')
jeju_dataset_day_df.to_csv('./data/jeju_pow3_day.csv')